# Monostate

> A singleton variation with multiple instances

The **Monostate** is a variation of the Singleton pattern that consists of putting all the state of an object into a static variable, but at the same time creating new objects is allowed; all of these new instances will all access the same state.

In this example we will create a `CEO` class. Most companies have a single CEO; we will allow them to call the initializer and construct a CEO however many times they want, but every single instance will be referring to the same CEO.

In order to accomplish this, our class will have a static variable (the shared state for all objects), and our initializer will assign this state to `__dict__`, a special attribute of every single object in Python that stores the object's writable attributes.

In [10]:
class CEO:
    __shared_state = {
        'name': 'Steve',
        'age': 55
    }
    
    def __init__(self):
        self.__dict__ = self.__shared_state

    def __str__(self):
        return f'{self.name} is {self.age} years old'


Let's see how it works:

In [11]:
ceo1 = CEO()
print(ceo1)

ceo1.age = 66

ceo2 = CEO()
ceo2.age = 77
print(ceo1)
print(ceo2)

ceo2.name = 'Tim'

ceo3 = CEO()
print(ceo1, ceo2, ceo3, sep=', ')

Steve is 55 years old
Steve is 77 years old
Steve is 77 years old
Tim is 77 years old, Tim is 77 years old, Tim is 77 years old


As you can see, modifying the attributes of `ceo2` also modifies the attributes of `ceo1`, because we're actually modifying the attributes of a single object that both variables refer to.

Essentially, whenever we create a new `CEO` object, what we're actually doing is creating a copy of the reference to the actual object.

we will now do a more generic approach by creating a `Monostate` class that other classes can inherit from in order to make them monostates. This `Monostate` class will also have a static shared state, and we will override the `__new__` method as follows:
1. We create a new object by calling the `__new__` method of the superclass. Because we're currently overriding `__new__`, we cannot do something like `super().__new__(cls, *args, **kwargs)` because we do not have an implicit `__class__` variable at this stage (needed for the zero-argument version of `super`), so we use the 2-argument form of `super` instead: the first argument is the class where the method is being defined, and the second is the actual class passed into `__new__`.
2. Like in the previous code, we make sure that the `__dict__` property of the object is referencing the static shared state.

In [3]:
class Monostate:
    _shared_state = {}

    def __new__(cls, *args, **kwargs):
        obj = super(Monostate, cls).__new__(cls, *args, **kwargs)
        obj.__dict__ = cls._shared_state
        return obj

And now we can inherit from `Monostate` to create a monostate class. We will override the initializer in order to define the state.

In [4]:
class CFO(Monostate):
    def __init__(self):
        self.name = ''
        self.money_managed = 0

    def __str__(self):
        return f'{self.name} manages ${self.money_managed}bn'

Now we can see how we can instantiate several copies of the object which will all share the same state:

In [8]:
cfo1 = CFO()
cfo1.name = 'Sheryl'
cfo1.money_managed = 1

print(cfo1)

cfo2 = CFO()
cfo2.name = 'Ruth'
cfo2.money_managed = 10
print(cfo1, cfo2, sep='\n')

Sheryl manages $1bn
Ruth manages $10bn
Ruth manages $10bn


However, it should be noted that this approach isn't practical. The recommended approach is probably by means of a decorator or a metaclass.